In [111]:
import numpy as np
import pandas as pd
import math
import random

random.seed(47)

In [2]:
# load data
combats = pd.read_csv('./Python/Data/combats.csv')
pokemons = pd.read_csv('./Python/Data/pokemon.csv')
tests = pd.read_csv('./Python/Data/tests.csv')

In [3]:
pokemons.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


In [89]:
print("attack range: %i - %i" % (pokemons.HP.min(), pokemons.HP.max()))
print("attack range: %i - %i" % (pokemons.Attack.min(), pokemons.Attack.max()))
print("attack range: %i - %i" % (pokemons.Defense.min(), pokemons.Defense.max()))

attack range: 1 - 255
attack range: 5 - 190
attack range: 5 - 230


In [71]:
# put range in vars
Hp = [pokemons.HP.min(), pokemons.HP.max()]
Atk = [pokemons.Attack.min(), pokemons.Attack.max()]
Def = [pokemons.Defense.min(), pokemons.Defense.max()]

In [6]:
pokemons['Type 1'].unique()

array(['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying'], dtype=object)

In [7]:
pokemons['Type 2'].unique().size

19

In [226]:
testdata = pokemons.loc[1:10,['HP','Attack','Defense','Name']]

In [227]:
testdata.values

array([[60, 62, 63, 'Ivysaur'],
       [80, 82, 83, 'Venusaur'],
       [80, 100, 123, 'Mega Venusaur'],
       [39, 52, 43, 'Charmander'],
       [58, 64, 58, 'Charmeleon'],
       [78, 84, 78, 'Charizard'],
       [78, 130, 111, 'Mega Charizard X'],
       [78, 104, 78, 'Mega Charizard Y'],
       [44, 48, 65, 'Squirtle'],
       [59, 63, 80, 'Wartortle']], dtype=object)

### Steps to take
1) Set amount of K clusters  
2) Assign each data point to closest centroid  
3) Recompute the cluster centroids(mean of all data points in cluster)  
4) back to step 2 and repeat

In [257]:
def KMeans(points, k = 3, max_iter = 10):
    # generate k random centroids
    centroids = centroid_gen(k)
    centroids = dict(centroids)
    print(centroids)
    cluster = {}
    
    iteration = 0
    while iteration < max_iter: #cen_diff != 0:
        for i in range(0,len(points)):
            distances = calc_dist(centroids, points[i])
            cluster[i] = np.min(distances)
            print(distances)
            print(points[i])
            print(centroids[0])
        iteration += 1
        print('\b')
    return cluster
#         while no change
#             for i in points:
            # assign data to closest cluster
        # compute distance and assign to closest cluster
            

In [112]:
# generate k random centroids
def centroid_gen(k):
    centroids = {}
    for c in range(0,k):
        rand_hp = random.randint(Hp[0],Hp[1])
        rand_atk = random.randint(Atk[0],Atk[1])
        rand_def = random.randint(Def[0],Def[1])
        centroids[c] = np.array([rand_hp, rand_atk, rand_def])
    return centroids

In [258]:
mest = KMeans(testdata.values , 2, 5)

{0: array([ 45,  43, 144]), 1: array([ 87, 167, 226])}
[149.093930124603, 340.9413439288348]
[60 62 63 'Ivysaur']
[ 45  43 144]
[142.08800090085018, 290.5873362691499]
[80 82 83 'Venusaur']
[ 45  43 144]
[166.80827317612278, 251.61279776672728]
[80 100 123 'Mega Venusaur']
[ 45  43 144]
[172.84675293449976, 386.2434465463459]
[39 52 43 'Charmander']
[ 45  43 144]
[150.93707298076242, 345.36068102782053]
[58 64 58 'Charmeleon']
[ 45  43 144]
[141.92251407017844, 294.74395668104887]
[78 84 78 'Charizard']
[ 45  43 144]
[178.12635964393365, 243.46868381785777]
[78 130 111 'Mega Charizard X']
[ 45  43 144]
[148.80188170853216, 280.9875442079239]
[78 104 78 'Mega Charizard Y']
[ 45  43 144]
[162.40997506310995, 366.43826219432924]
[44 48 65 'Squirtle']
[ 45  43 144]
[147.14618581533128, 327.4629750063356]
[59 63 80 'Wartortle']
[ 45  43 144]

[149.093930124603, 340.9413439288348]
[60 62 63 'Ivysaur']
[ 45  43 144]
[142.08800090085018, 290.5873362691499]
[80 82 83 'Venusaur']
[ 45  43 144]


In [251]:
mest

{0: 89.86656775464388,
 1: 138.26062346163494,
 2: 198.48425630260957,
 3: 67.86015030929418,
 4: 86.82741502543998,
 5: 134.08579343092242,
 6: 216.30533973991487,
 7: 156.00961508830153,
 8: 78.23042886243178,
 9: 105.8536725862641}

In [256]:
mest

{0: 235.5610324310878,
 1: 177.11295830627412,
 2: 132.89469515371937,
 3: 251.9206224190469,
 4: 240.58678267934837,
 5: 182.10436568078208,
 6: 125.2078272313676,
 7: 166.73931749890306,
 8: 247.1214276423637,
 9: 220.5810508633958}

In [259]:
print(math.sqrt(7147))

84.53993139339539


In [137]:
testar = np.array([11, 112, 173])

In [178]:
# calculate Euclidean distance
def calc_dist(cen, p):
    distances = []
    for i in cen:
        distance = math.sqrt( sum((cen[i]-p[0])**2 + (cen[i]-p[1])**2 + (cen[i]-p[2])**2))
        distances.append(distance)
    return distances

In [179]:
dista = calc_dist(ceno,testar)
np.min(dista)

218.55891654197043

In [147]:
(ceno[0][0] - testar[0])**2 + (ceno[0][0] - testar[0])**2

96800

In [150]:
ceno[0]

array([231,  85, 150])

In [152]:
testar[0]

11